In [1]:
import warnings 
warnings.filterwarnings("ignore")
import tensorflow as tf
import keras
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sn 
%matplotlib inline 
from sklearn.model_selection import train_test_split 
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import log_loss
from sentence_transformers import SentenceTransformer 
# import torch 
# import os
import random 
from sklearn.metrics import accuracy_score,confusion_matrix, f1_score
# import urduhack #for pre processing Urdu Text
np.random.seed(42)
tf.random.set_seed(42)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
random.seed(42)
#other required libraries will be imported at the time of requirement 

2023-03-04 14:51:03.915469: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 14:51:04.029505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-04 14:51:04.033592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-04 14:51:04.033606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
#reading the data 
df_train=pd.read_csv("Train_Clean.csv")
df_test=pd.read_csv("Test_Clean.csv")
print("############################################\n Training Data\n")
display(df_train.head(10))
print("############################################\n Testing Data\n")
display(df_test.head(10))

############################################
 Training Data



,anger,disgust,fear,sadness,surprise,happiness,neutral,Sentences,word_count,char_count,word_density
0,0,0,0,1,1,0,0,محبت پردے میں نفرت کرنےوالو,5,23,0.208333
1,0,0,0,0,0,1,0,حامد میر خوشی اس کا ذمہ دار حکومت خوش,9,29,0.300000
2,0,0,0,0,0,0,1,اس نئے سال شام سپاہی قاتل لارنس شکل میں شعبے ب...,135,526,0.256167
3,0,0,0,0,0,0,1,کچھ اس طرح سے فلم جا سکتے لیکن میں سب سے یقینا...,117,435,0.268349
4,0,0,0,0,0,0,1,رون ہاورڈ اس ایڈیٹرز صرف اتنا کرنا جس کا تھا ا...,84,304,0.275410
5,0,0,0,0,0,0,0,نتائج دنیا کو حیرت میں ڈال سکتی ھے نتائج دس اق...,19,69,0.271429
6,0,0,0,0,1,0,0,ہم غصے خراب موڈ خوشی غمی سب میں چپلی کباب,10,32,0.303030
7,0,0,0,1,1,0,0,اب ایسی حیرت وارفتگی کو کیا کہیے دعا کو ہاتھ ا...,14,52,0.264151
8,0,0,0,1,1,0,0,آپ اپنی آنکھوں سے نفرت پٹی اتاریں شائد اندازہ ...,13,55,0.232143
9,0,0,0,0,1,0,0,حیرت,1,4,0.200000


############################################
 Testing Data



,anger,disgust,fear,sadness,surprise,happiness,neutral,Sentences
0,0,0,0,1,0,0,0,بچھڑنے والوں کا کیسے نہ غم کیا جائے بوجھ ایسا ...
1,0,0,0,0,0,0,1,اس خوفناک اذیت تشدد گندگی میں بدترین ڈی تعاون ...
2,0,0,0,0,0,0,1,تمام ناقص جاز رپ آف افسوس میری وقت پسندیدہ ڈائ...
3,0,0,0,0,0,0,1,اس فلم ساتھ راجر ایوری کوئنتن تارتانتینو سے ہد...
4,0,0,0,0,0,0,1,جنگلی جانوروں غیر ملکی خطوں غیر معمولی ثقافتوں...
5,0,0,0,0,0,0,1,لڑکی سیریل بلاتکاری روشندان ذریعے اس گھور رہا ...
6,0,0,0,1,1,0,0,سر خدا کا واسطہ راولپنڈی پے توجہ دلوائیں راجہ ...
7,0,0,0,0,0,0,1,عزیز مجھے میں کہاں سے آغاز کروں والد بھی لڑکی ...
8,0,0,0,1,0,0,0,بہت آسان ھائی ذرا دماغ کو نفرت بغض عینک اتار ک...
9,0,0,0,0,0,0,1,میں نے سپائک ان فلم مختلف عنوان منتخب کیا تھا ...


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Flatten, Dense, Embedding

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [6]:
#Splitting the dataset into 80:20 ratio 
x_train_val=df_train["Sentences"]
y_train_val=df_train[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
x_test=df_test["Sentences"]
y_test=df_test[["anger","disgust","fear","sadness","surprise","happiness","neutral"]]
x_train,x_val,y_train,y_val=train_test_split(x_train_val,y_train_val,test_size=0.2,random_state=42)

In [7]:
#shape of training data 
print("Shape of train data is\nx_train------>",x_train.shape)
print("y_train------>",y_train.shape)
print("Shape of test data is\nx_test------>",x_test.shape)
print("y_test------>",y_test.shape)
print("Shape of Validation data is\nx_val------>",x_val.shape)
print("y_val------>",y_val.shape)

Shape of train data is
x_train------> (6240,)
y_train------> (6240, 7)
Shape of test data is
x_test------> (1950,)
y_test------> (1950, 7)
Shape of Validation data is
x_val------> (1560,)
y_val------> (1560, 7)


In [8]:
max_len = 1500
max_features = 5000

In [9]:
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(x_train_val.values)

x_train_converted = tokenizer.texts_to_sequences(x_train.values)
x_train_converted = pad_sequences(x_train_converted, maxlen=max_len)

x_test_converted = tokenizer.texts_to_sequences(x_test.values)
x_test_converted = pad_sequences(x_test_converted, maxlen=max_len)
x_val_converted = tokenizer.texts_to_sequences(x_val.values)
x_val_converted = pad_sequences(x_val_converted, maxlen=max_len)

In [10]:
print('The shape of train data :', x_train_converted.shape)
print('The shape of target of the training :', y_train.shape)
print('The shape of test data :', x_test_converted.shape)
print("The shaoe of target of the test: ", y_test.shape)
print("The shape of validation data: ",x_val_converted.shape)
print("The shape of target of validation : ",y_val.shape)

The shape of train data : (6240, 1500)
The shape of target of the training : (6240, 7)
The shape of test data : (1950, 1500)
The shaoe of target of the test:  (1950, 7)
The shape of validation data:  (1560, 1500)
The shape of target of validation :  (1560, 7)


In [18]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Conv1D, BatchNormalization, LSTM, Bidirectional
from keras.layers import MaxPooling1D
from keras.layers import Input, Flatten, Dropout
from keras.layers import Activation
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
from keras.utils import to_categorical
import tensorflow as tf
import keras
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import time
from pathlib import Path
import random 
np.random.seed(42)
random.seed(42)
import warnings 
warnings.filterwarnings("ignore")

In [19]:
from tensorflow.keras.layers import SpatialDropout1D
model = Sequential()
model.add(Embedding(1500, 100,input_length = x_train_converted.shape[1]))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.3,recurrent_dropout=0.3)))
model.add(Bidirectional(LSTM(64,return_sequences=True,dropout=0.3,recurrent_dropout=0.3)))
model.add(LSTM(32, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(7,activation='softmax'))
model.compile(loss='categorical_crossentropy',metrics='accuracy', optimizer='Adam')
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1500, 100)         150000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 1500, 100)        0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 1500, 256)        234496    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 1500, 128)        164352    
 nal)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 32)                20608     
                                                      

In [20]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5,min_lr=1e-05,verbose=1)
early_stoping = EarlyStopping(monitor="val_loss",min_delta=0,patience=5,verbose=1,mode="auto", baseline=None,restore_best_weights=True)
histroy = model.fit(x_train_converted,y_train, validation_data=(x_val_converted,y_val),epochs=200,callbacks=[reduce_lr,early_stoping],verbose=1,batch_size=34)

Epoch 1/200


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_1/embedding_1/embedding_lookup' defined at (most recent call last):
    File "/home/shirish/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/shirish/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/shirish/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/shirish/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/shirish/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/shirish/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/home/shirish/anaconda3/lib/pyt